##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Search re-ranking using Gemini embeddings

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   The user will make a search query.
2.   You will use Wikipedia API to return the relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity.
4.   The most relevant search result will be returned as the final answer.

> The non-source code materials in this notebook are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0, https://creativecommons.org/licenses/by-sa/4.0/legalcode.

## Setup


In [2]:
%pip install -q -U "google-genai>=1.0.0"


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q wikipedia


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: The [`wikipedia` package](https://pypi.org/project/wikipedia/) notes that it was "designed for ease of use and simplicity, not for advanced use", and that production or heavy use should instead "use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python)".

In [4]:
import json
import textwrap

from google import genai
from google.genai import types

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import numpy as np

from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [5]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Initialize the client with your API key
client = genai.Client(api_key=GOOGLE_API_KEY)

ModuleNotFoundError: No module named 'google.colab'

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

### Define the search function

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-2.0-flash` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [ ]:
def wikipedia_search(search_queries: list[str]) -> list[str]:
  """Search wikipedia for each query and summarize relevant docs."""
  n_topics=3
  search_history = set() # tracking search history
  search_urls = []
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)

    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue

      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history

      try:
        # extract the relevant data by using `gemini-2.0-flash` model
        page = wikipedia.page(search_term, auto_suggest=False)
        url = page.url
        print(f"Information Source: {url}")
        search_urls.append(url)
        page = page.content
        
        # Using the client to generate content
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=textwrap.dedent(f"""\
                Extract relevant information
                about user's query: {query}
                From this source:

                {page}

                Note: Do not summarize. Only Extract and return the relevant information
            """)
        )

        urls = [url]
        if response.candidates and response.candidates[0].citation_metadata:
          extra_citations = response.candidates[0].citation_metadata.citation_sources
          extra_urls = [source.url for source in extra_citations]
          urls.extend(extra_urls)
          search_urls.extend(extra_urls)
          print("Additional citations:", response.candidates[0].citation_metadata.citation_sources)
        
        try:
          text = response.text
        except (ValueError, AttributeError):
          pass
        else:
          summary_results.append(text + "\n\nBased on:\n  " + ',\n  '.join(urls))

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")
        continue

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')
        continue
        
      except Exception as e:
        print(f'Error processing {search_term}: {str(e)}')
        continue

  print(f"Information Sources:")
  for url in search_urls:
    print('    ', url)

  return summary_results


In [ ]:
example = wikipedia_search(["What are LLMs?"])

Here is what the search results look like:

In [ ]:
from IPython.display import display

for e in example:
  display(to_markdown(e))

### Pass the tools to the model

In the new Google GenAI SDK v1.0.0+, function calling is handled by registering your tools with the client and then providing them when sending messages. The client will handle the extraction of schema from function signatures, and the model may return a function call object when it wants to call the function.

The functions need to have proper type annotations for parameters and return values to work correctly.

In [ ]:
# Define the generation config
generation_config = types.GenerationConfig(temperature=0.6)

# You'll use this config when creating the chat session
model = genai.GenerativeModel(
    'gemini-2.0-flash',
    tools=[wikipedia_search],
    generation_config=generation_config)

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [ ]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

user: Tell me about Cricket World cup 2023 winners.

function_call: wikipedia_search(['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?'])

The search function will return a list of article summaries, use these to
answer the  user's question.

Here is the user's query: {query}
"""

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

## Enable automatic function calling and call the API

Now start a new chat with `enable_automatic_function_calling=True`. With it enabled, the `genai.ChatSession` will handle the back and forth required to call the function, and return the final response:

In [ ]:
# Create a chat session with the client
chat = client.chats.create(
    model='gemini-2.0-flash',
    tools=[{"function_declarations": [{"name": "wikipedia_search"}]}],
    generation_config=generation_config
)

query = "Explain how deep-sea life survives."

# Send the message to the chat
res = chat.send_message(message=instructions.format(query=query), tool_config={
    "function_calling_config": {
        "mode": types.FunctionCallingMode.AUTO,
        "allowed_function_names": ["wikipedia_search"]
    }
}, tools=[wikipedia_search])

In [ ]:
to_markdown(res.text)

Check for additional citations:

In [ ]:
res.candidates[0].citation_metadata or 'No citations found'

That looks like it worked. You can go through the chat history to see the details of what was sent and received in the function calls:

In [ ]:
for content in chat.history:
  print(f'{content.role} -> ')
  
  if hasattr(content, 'parts') and content.parts:
    part = content.parts[0]
    if hasattr(part, 'function_call') and part.function_call:
      print(f"Function Call: {part.function_call.name}")
      print(f"Arguments: {json.dumps(part.function_call.args, indent=2)}")
    elif hasattr(part, 'function_response') and part.function_response:
      print(f"Function Response: {part.function_response.name}")
      print(f"Response: {json.dumps(part.function_response.response, indent=2)}")
    else:
      print(content.parts[0].text if hasattr(content.parts[0], 'text') else content.parts[0])
  print('---' * 20)


In the chat history you can see all the steps of the conversation:

1. The user sent the query.
2. The model replied with a function call to `wikipedia_search` with a number of relevant searches.
3. Because you configured the tool and provided it when sending the message, the client handled executing the search function and returning the list of article summaries to the model.
4. Following the instructions in the prompt, the model generated a final answer based on those summaries.


## [Optional] Manually execute the function call

If you want to understand what happened behind the scenes, this section executes the `FunctionCall` manually to demonstrate.

In [ ]:
# Create a new chat session without automatic function calling
chat = client.chats.create(
    model='gemini-2.0-flash',
    tools=[{"function_declarations": [{"name": "wikipedia_search"}]}],
    generation_config=generation_config
)

In [ ]:
# Send a message to the chat without automatic function calling
result = chat.send_message(message=instructions.format(query=query), tools=[wikipedia_search])

Initially the model returns a FunctionCall:

In [ ]:
# Extract the function call from the response
if result.candidates and len(result.candidates) > 0 and result.candidates[0].content.parts:
    function_call = result.candidates[0].content.parts[0].function_call
    if function_call:
        print(json.dumps({
            "name": function_call.name,
            "args": function_call.args
        }, indent=2))
    else:
        print("No function call found in the response.")
else:
    print("Unexpected response format.")

In [ ]:
# Access the function name
if 'name' in locals() and function_call and function_call.name:
    print(function_call.name)
else:
    print("Function name not available")

Call the function with generated arguments to get the results.

In [ ]:
# Execute the function with the arguments from the function call
if function_call and hasattr(function_call, 'args'):
    summaries = wikipedia_search(**function_call.args)
else:
    print("Could not execute function, missing function call or arguments.")
    summaries = []

Now send the `FunctionResult` to the model.

In [ ]:
# Create a function response message
response = chat.send_message(
    message=types.Content(
        parts=[types.Part(
            function_response=types.FunctionResponse(
                name="wikipedia_search",
                response={
                    "result": summaries
                }
            )
        )]
    )
)

to_markdown(response.text)

## Re-ranking the search results

Helper function to embed the content:

In [ ]:
def get_embeddings(content: list[str]) -> np.ndarray:
  # Using client.models.embed_content for embeddings
  response = client.models.embed_content(
      model="embedding-001",  # Note: model name format has changed
      contents=content,
      task_type="SEMANTIC_SIMILARITY"
  )
  
  # Extract embeddings from the response
  embds = [embedding.values for embedding in response.embeddings]
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

Please refer to the [embeddings guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement cosine similarity as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating cosine similarity is esentially same as calculating their dot product score.

In [ ]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [ ]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

Calculate similarity score:

In [ ]:
sim_value = dot_product(search_res, embedded_query)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
print(summaries[np.argmax(sim_value)])

### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [Gao et al](https://arxiv.org/abs/2212.10496) the objective here is to generate a template answer to the user's query using `gemini-2.0-flash`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [ ]:
# Generate hypothetical answer using the client
res = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}"""
)

to_markdown(res.text)

Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [ ]:
hypothetical_ans = get_embeddings([res.text])

Calculate similarity scores to rank the search results

In [ ]:
sim_value = dot_product(search_res, hypothetical_ans)

In [ ]:
sim_value

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
to_markdown(summaries[np.argmax(sim_value)])

You have now created a search re-ranking engine using Gemini embeddings with the Google GenAI SDK v1.0.0+!

In this notebook, you've learned how to:
1. Set up and use the new Google GenAI client
2. Use function calling to access the Wikipedia API
3. Embed content using the new embedding API format
4. Re-rank search results using similarity metrics

## Next steps

I hope you found this example helpful! Check out more examples in the [Gemini Guide](https://github.com/google-gemini/gemini-guide/) to learn more.